In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from covis.utils import get_project_root

pd.set_option("display.max_column", None)
sns.set_style("whitegrid")
sns.set_palette("tab20")

In [ ]:
# jpath = get_project_root() / "output/eer_dict.json"
# with open(jpath) as f:
#     eer = json.load(f)

# rows = [4,5]
# rows.extend(range(76,86))

# weekly_reg_covid19_2020 = pd.read_excel(
#     get_project_root() / "data/publishedweek532020.xlsx",
#     sheet_name="Covid-19 - Weekly registrations",
#     skiprows=lambda x: x not in rows,
#     usecols=lambda x: x not in ["Unnamed: 1"]
# )

# weekly_reg_covid19_2020 = weekly_reg_covid19_2020.set_index("Week number").transpose()
# weekly_reg_covid19_2020.rename(columns={"covid19 deaths, all ages":"covid19 deaths, all ages (2020)"}, inplace=True)
# weekly_reg_covid19_2020.rename(columns=eer, inplace=True)

# for col in [col for col in weekly_reg_covid19_2020.columns if col not in ["Week ended"]]:
#     weekly_reg_covid19_2020[col] = weekly_reg_covid19_2020[col].astype(int)

# weekly_reg_covid19_2020["total"] = weekly_reg_covid19_2020.sum(axis=1)

# print(f"covid19 deaths 2020\n{weekly_reg_covid19_2020.shape}")
# weekly_reg_covid19_2020.head(50)

### will look at the period march 2020 - may 2020 (before advice / lockdowns occur)

In [ ]:
rows = [4,5]
rows.extend(range(11,31))

total = pd.read_excel(
    get_project_root() / "data/publishedweek532020.xlsx",
    # sheet_name="UK - Covid-19 - Weekly reg",
    sheet_name="Covid-19 - Weekly registrations",
    skiprows=lambda x: x not in rows,
    # usecols=lambda x: x not in ["Unnamed: 1"]
)

for i in range(1,21):
    total["Week number"][i] = total["Unnamed: 1"][i]

total.drop(columns=["Unnamed: 1"], inplace=True)

In [ ]:
# total

In [ ]:
total = total.set_index("Week number").transpose()

In [ ]:
total.dtypes

In [ ]:
# cast to integers
for col in [col for col in total.columns if col not in ["Week ended"]]:
    total[col] = total[col].astype(int)

In [ ]:
total_mam = total[total["Week ended"].between("2020-03-01", "2020-05-31")].copy()

In [ ]:
total_mam

In [ ]:
sns.barplot(
    total_mam
);

In [ ]:
total_mam.sum()

In [ ]:
def get_age_data_2020(rows, start, end):
    df = pd.read_excel(
        get_project_root() / "data/publishedweek532020.xlsx",
        sheet_name="Covid-19 - Weekly registrations",
        skiprows=lambda x: x not in rows
    )

    for i in range(1,21):
        df["Week number"][i] = df["Unnamed: 1"][i]

    df.drop(columns=["Unnamed: 1"], inplace=True)

    df = df.set_index("Week number").transpose()
    
    for col in [col for col in df.columns if col not in ["Week ended"]]:
        df[col] = df[col].astype(int)

    df = df[df["Week ended"].between(f"{start}", f"{end}")]

    return df

In [ ]:
rows = [4,5]
rows.extend(range(33,53))

males_mam = get_age_data_2020(rows, "2020-03-01", "2020-05-31")

In [ ]:
males_mam

In [ ]:
# males_mam.sum()

In [ ]:
rows = [4,5]
rows.extend(range(55,75))

females_mam = get_age_data_2020(rows, "2020-03-01", "2020-05-31")

In [ ]:
females_mam

In [ ]:
# females_mam.sum()

In [ ]:
males_mam = males_mam.melt(
    "Week ended",
    var_name="age",
    value_name="deaths"
)

males_mam = males_mam.groupby("age").sum()
males_mam["sex"] = ["male"]*len(males_mam)

In [ ]:
females_mam = females_mam.melt(
    "Week ended",
    var_name="age",
    value_name="deaths"
)

females_mam = females_mam.groupby("age").sum()
females_mam["sex"] = ["female"]*len(females_mam)

In [ ]:
males_mam

In [ ]:
females_mam

In [ ]:
age_sex = pd.concat(
    [males_mam, females_mam]
)

In [ ]:
age_sex.rename(index={"<1":"0", "1-4":"01-04", "5-9":"05-09"}, inplace=True)

In [ ]:
age_sex.sort_index(inplace=True)

In [ ]:
fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(111)

sns.barplot(
    age_sex,
    x=age_sex.index,
    y="deaths",
    hue="sex",
    errorbar=None,
    ax=ax
)

ax.set_xlabel("age group")
fig.suptitle("registered deaths due to covid-19 (march 2020 - may 2020), england & wales")
fig.tight_layout();

In [ ]:
# fig.savefig(
#     get_project_root() / "figures/age_sex_march_may_2020.png"
# )